Magnetite Oxides
================



The magnetite data is expressed in elemental wt% rather than oxide wt% since oxygen was measured. To convert into oxide wt%:

1.  Find moles of each component
2.  Determine how much oxygen remains after the moles oxide components of all elements except Fe are computed.
3.  Partition this remaining O into FeO and Fe2O3.
4.  Convert moles of oxide to mass of oxide
5.  Normalize result to sum to 100% (i.e. closure to 100%).



In [1]:
import sys
import os
sys.path.insert(0,os.path.join("..","XRF_to_PEM"))
from composition_processor import Ar,Molecule
import pandas as pd
import numpy as np

# Load elements df.
df = pd.read_csv("magnetites.csv")
# Filter for magnetite in minerals of interest.
minerals_of_interest = ["srp","clc","early-clc"]
df = df[df["Mineral"].apply(lambda x : x in minerals_of_interest)]
# Declare the elements present in the data.
analysed_elements = ["Si","Ti","Mn","Cr","Ni","O","Mg","Fe"]
# Convert mass to moles of each element.
mol_df = df[analysed_elements].apply(lambda col : col/Ar[col.name])
# Clip negatives.
mol_df = np.clip(mol_df,0,100)
# Set index to spot names.
mol_df.index = df["Comment"]
# Declare number of own atoms and oxygen atoms required to form each element's standard oxide.
std_oxides = {"Si":(1,2),
              "Ti":(1,2),
              "Mn":(1,1),
              "Cr":(2,3),
              "Ni":(1,1),
              "Mg":(1,1)}
# Iterate through elements in the dataframe.
for elem in mol_df:
    # Check whether the element has a corresponding entry in the manually written translation dictionary between element and element oxide (i.e. just not for Fe).
    if elem in std_oxides:
        # Construct the chemical composition string describing the element oxide.
        str_multiple = lambda n : str(n) if n != 1 else ""
        oxide_str = elem + str_multiple(std_oxides[elem][0]) + "O" + str_multiple(std_oxides[elem][1])
        # Compute moles of oxides of moles of each element.
        mol_df[oxide_str] = mol_df[elem] / std_oxides[elem][0]
        # Compute moles of oxygen removed by the moles of oxide.
        mol_df["O"] -= std_oxides[elem][1] * mol_df[oxide_str]
# Compute moles of Fe2O3 (which all O outside of FeO (which has the same number of moles as Fe) goes into).
mol_df["Fe2O3"] = mol_df["O"] - mol_df["Fe"]
# Compute moles of FeO (= moles of Fe2+ = FeTot - Fe3+)
mol_df["FeO"] = mol_df["Fe"] - 2 * mol_df["Fe2O3"]
# Remove the element-only data.
mol_df = mol_df.drop(analysed_elements,axis=1)
# Convert oxide moles to oxide mass.
mass_df = mol_df.apply(lambda col : col * Molecule(col.name).Mr())
# Normalize to 100%.
mass_df = mass_df.div(mass_df.sum(axis=1),axis=0) * 100

## Composition of Magnetite in Partially vs Heavily Serpentinized Samples



Do the magnetites in partially vs heavily serpentinized rocks have significantly different compositions?

Can first check differences in Cr contents (the main "contaminant"), and then apply a chi2 test.



### Cr Comparison



In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Dictionary to map sample name to recorded host mineral.
sample_map = dict(zip(df["Comment"],df["Mineral"]))

classification = {"Partially":["M04","M08"],
                  "Heavily":["06C","M01"]}
dfs = dict()
cr2o3s = []
for name,samples in classification.items():
    filtered_serp_df = mass_df.copy()[[any([x in idx for x in samples]) for idx in mass_df.index]]
    filtered_serp_df["group"] = name
    filtered_serp_df["host"] = [sample_map[sample] for sample in filtered_serp_df.index]
    cr2o3 = filtered_serp_df["Cr2O3"]
    print(name + " serp Cr2O3 wt%% (n=%u)\n" % len(filtered_serp_df),"Mean: %.2f" % np.mean(cr2o3),"Stdev: %.2f" % np.std(cr2o3))
    cr2o3s.append(cr2o3)
    dfs[name] = filtered_serp_df

plt.boxplot(cr2o3s)
plt.gca().set_xticks([1,2],["P","H"])
plt.ylabel("Cr wt%")
plt.show()

None